In [2]:
import reg_mri
import os
from glob import glob
from utils import compute_mean_dice
import nibabel as nib
from scipy.spatial.distance import dice
import numpy as np
import itk
import SimpleITK as sitk
import scipy.ndimage
import scipy
import matplotlib.pyplot as plt
from transforms_dict import getRegistrationEvalInverseTransformForMRI, SaveTransformForMRI
from tqdm import tqdm
import monai
import subprocess
from monai.networks.blocks import Warp
import torch
import pandas as pd
import utils

In [3]:
def getRegionsAsCSV_Physical(mri, output):
    command = "ImageMath 3 " + str(output) + " LabelStats " + str(mri) + " " + str(mri)
    subprocess.call(command.split(" "))

In [4]:
def getRegionsAsCSV_Voxel(mask):    
    img = nib.load(mask).get_fdata().astype(np.uint8)
    regions = regionprops(img)
    df = pd.DataFrame()
    for props in regions:
        x,y,z = props.centroid
        x = x #+ 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
        y = y #+ 1
        z = z #+ 1
        t = 0
        label = props.label
        mass = props.area
        volume = props.area
        count = len(props.coords)
        row = {
            'x': x,
            'y': y,
            'z': z,
            't': t,
            'label': label,
            'mass': mass,
            'volume': volume,
            'count': count,
        }
        temp_df = pd.DataFrame(data=row, index=[0])
        df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [5]:
def compute_csv_distance(csv1, csv2):
    df1 = pd.read_csv(csv1, sep=',', header=0)
    df2 = pd.read_csv(csv2, sep=',', header=0)
    TREs = []
    for i in df1['label']:
        ind1 = np.where(df1['label'] == i)[0]
        ind2 = np.where(df2['label'] == i)[0]
        if len(ind1) == 1 and len(ind2) == 1:
            x1 = df1['x'][ind1[0]]
            x2 = df2['x'][ind2[0]]            
            y1 = df1['y'][ind1[0]]
            y2 = df2['y'][ind2[0]]            
            z1 = df1['z'][ind1[0]]
            z2 = df2['z'][ind2[0]]  
            
            TRE = np.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2)+(z1-z2)*(z1-z2))
            TREs.append(TRE)    
    return TREs

In [6]:
atlas_name = "dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz"
atlas_landmark = "dataset2/Atlas/Deformable_Feminad_Landmarks_template0_sameaffine.nii.gz"
atlas_landmark_torch = torch.from_numpy(nib.load(atlas_landmark).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)

landmarks      = sorted(glob(os.path.join('dataset2', 'Feminad', 'Landmarks_Mask_Resample', "*.nii.gz")))

ants_mris      = os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm')
aff_ants_warps = os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm_Deformable')
def_ants_warps = os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm_Deformable')

deep_mris      = os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm_Affine_Template')
aff_deep_warps = os.path.join('dataset2', 'Feminad', 'MRI_N4_Resample_Norm_Affine_Template')
def_deep_warps = os.path.join('output', 'Feminad', 'DL_Localnet_Reg', 'DeformableWarp')

In [7]:
def ants_apply_transform(inp, ref, mat):
    out = "tmp.nii.gz"
    jacobian_command = "antsApplyTransforms -d 3 -n Linear -i " + str(inp) + " -o " + str(out) + " -r " + str(ref) + " -t " + str(mat)
    subprocess.call(jacobian_command.split(" "))
    return nib.load(out)

In [27]:
affine = nib.load(atlas_name).affine
header = nib.load(atlas_name).header

warp = Warp("bilinear", "border")
warp_nearest = Warp("nearest", "border")

for i, landmark_name in enumerate(landmarks):
    name = landmark_name.split('/')[-1].split('.')[0].split('_')
    name = name[1] + '_' + name[2]
    print(name)
    
    outfolder = os.path.join('output', 'Feminad', 'Landmarks_Check')
    
    aff_ants_warp = glob(os.path.join(aff_ants_warps,  '*' + name + '*.mat'))[0]
    aff_ants_warp_physical = utils.get_warp_from_mat_file("dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz", aff_ants_warp)          
    aff_ants_warp_physical_name = "tmp_aff_ants_warp_physical.nii.gz"
    nib.save(aff_ants_warp_physical, aff_ants_warp_physical_name)
    aff_ants_warp_voxel = utils.convert_warp_physical_to_voxel(aff_ants_warp_physical)
    
    def_ants_warp = sorted(glob(os.path.join(def_ants_warps,  '*' + name + '*Warp.nii.gz')))[1]
    def_ants_warp_physical = nib.load(def_ants_warp)    
    def_ants_warp_voxel = utils.convert_warp_physical_to_voxel(def_ants_warp_physical)
    
    aff_deep_warp = glob(os.path.join(aff_deep_warps,  '*' + name + '*.mat'))[0]        
    aff_deep_warp_physical = utils.get_warp_from_mat_file("dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz", aff_deep_warp)          
    aff_deep_warp_physical_name = "tmp_aff_deep_warp_physical.nii.gz"
    nib.save(aff_deep_warp_physical, aff_deep_warp_physical_name)
    aff_deep_warp_voxel = utils.convert_warp_physical_to_voxel(aff_deep_warp_physical)
        
    def_deep_warp = glob(os.path.join(def_deep_warps,  '*' + name + '*.nii.gz'))[0]
    def_deep_warp_voxel = nib.load(def_deep_warp)
    
    ants_mri = glob(os.path.join(ants_mris,  '*' + name + '*.nii.gz'))[0]    
    ants_mri_nib = nib.load(ants_mri)
    
    deep_mri = glob(os.path.join(deep_mris,  '*' + name + '*.nii.gz'))[0]    
    deep_mri_nib = nib.load(ants_mri)
    
    ants_landmark = nib.load(landmark_name)
    
    # 1) ANTS + ApplyTransform
    mri_affine_ants_physical = ants_apply_transform(ants_mri, atlas_name, aff_ants_warp)
    outname = outfolder + "/" + name + "_ants_physical_affine.nii.gz"
    nib.save(mri_affine_ants_physical, outname)
    
    mri_deformable_ants_physical = ants_apply_transform(outname, atlas_name, def_ants_warp)
    outname = outfolder + "/" + name + "_ants_physical_deformable.nii.gz"
    nib.save(mri_deformable_ants_physical, outname)
    
    # 2) ANTS + ApplyTransform with Mat2DDF
    mri_affine_ants_mat2ddf_physical = ants_apply_transform(ants_mri, atlas_name, aff_ants_warp_physical_name)
    outname = outfolder + "/" + name + "_ants_physical_mat2ddf_affine.nii.gz"
    nib.save(mri_affine_ants_mat2ddf_physical, outname)
    
    mri_deformable_ants_mat2ddf_physical = ants_apply_transform(outname, atlas_name, def_ants_warp)
    outname = outfolder + "/" + name + "_ants_physical_mat2ddf_deformable.nii.gz"
    nib.save(mri_deformable_ants_mat2ddf_physical, outname)
    
    ## 3) ANTS + Warp with Mat2DDF+ ConvertPhys2Voxel    
    ants_mri_warp = torch.from_numpy(nib.load(ants_mri).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)
    aff_ants_warp_voxel_torch = torch.from_numpy(aff_ants_warp_voxel.get_fdata())    
    mri_affine_ants_voxel = warp(ants_mri_warp, aff_ants_warp_voxel_torch)
    outname = outfolder + "/" + name + "_ants_voxel_affine.nii.gz"
    mri_affine_ants_voxel_tonib = nib.Nifti1Image(mri_affine_ants_voxel.squeeze().numpy(), affine, header)
    nib.save(mri_affine_ants_voxel_tonib, outname)
    
    ants_mri_affine_warp = torch.from_numpy(nib.load(outname).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)     
    def_ants_warp_voxel_torch = torch.from_numpy(def_ants_warp_voxel.get_fdata())
    mri_deformable_ants_voxel = warp(ants_mri_affine_warp, def_ants_warp_voxel_torch)
    outname = outfolder + "/" + name + "_ants_voxel_deformable.nii.gz"
    mri_deformable_ants_voxel_tonib = nib.Nifti1Image(mri_deformable_ants_voxel.squeeze().numpy(), affine, header)
    nib.save(mri_deformable_ants_voxel_tonib, outname)       
    
    ## 4) ANTS Affine + DL Deformable from raw MRI
    
    deep_mri_warp = torch.from_numpy(nib.load(ants_mri).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)
    aff_deep_warp_voxel_torch = torch.from_numpy(aff_deep_warp_voxel.get_fdata())    
    mri_affine_deep_voxel = warp(deep_mri_warp, aff_deep_warp_voxel_torch)
    outname = outfolder + "/" + name + "_deep_voxel_affine.nii.gz"
    mri_affine_deep_voxel_tonib = nib.Nifti1Image(mri_affine_deep_voxel.squeeze().numpy(), affine, header)
    nib.save(mri_affine_deep_voxel_tonib, outname)
    
    deep_mri_affine_warp = torch.from_numpy(nib.load(outname).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)     
    def_deep_warp_voxel_torch = torch.from_numpy(def_deep_warp_voxel.get_fdata()).permute(3,4,0,1,2)
    mri_deformable_deep_voxel = warp(deep_mri_affine_warp, def_deep_warp_voxel_torch)
    outname = outfolder + "/" + name + "_deep_voxel_affine_deformable.nii.gz"
    mri_deformable_deep_voxel_tonib = nib.Nifti1Image(mri_deformable_deep_voxel.squeeze().numpy(), affine, header)
    nib.save(mri_deformable_deep_voxel_tonib, outname)    
    
    ## 5) DL Deformable from Affine MRI
    
    deep_mri_affine_warp = torch.from_numpy(nib.load(deep_mri).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)     
    def_deep_warp_voxel_torch = torch.from_numpy(def_deep_warp_voxel.get_fdata()).permute(3,4,0,1,2)
    mri_deformable_deep_voxel = warp(deep_mri_affine_warp, def_deep_warp_voxel_torch)
    outname = outfolder + "/" + name + "_deep_voxel_deformable_fromaffine.nii.gz"
    mri_deformable_deep_voxel_tonib = nib.Nifti1Image(mri_deformable_deep_voxel.squeeze().numpy(), affine, header)
    nib.save(mri_deformable_deep_voxel_tonib, outname)    
    break

monai.networks.blocks.Warp: Using PyTorch native grid_sample.


2_6516


In [63]:
affine = nib.load(atlas_name).affine
header = nib.load(atlas_name).header

for i, mri_filename in enumerate(mris):
    filename = mri_filename.split('/')[-1].split('_')[-1]
    print(filename)
    
    affine_warp = torch.from_numpy(nib.load(affine_warps[i]).get_fdata()).permute(3,4,0,1,2)
    deformable_warp = torch.from_numpy(nib.load(def_warps[i]).get_fdata()).permute(3,4,0,1,2)
        
    mri = torch.from_numpy(nib.load(mris[i]).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)
    
    affine_mri = warp(mri, affine_warp)
    affine_mri_nib = nib.Nifti1Image(affine_mri.squeeze().numpy(), affine, header)
    nib.save(affine_mri_nib, 'output/Feminad/Test_Landmarks/Affine/Affine_' + filename)
    
    deformable_mri = warp(affine_mri, deformable_warp)
    deformable_mri_nib = nib.Nifti1Image(deformable_mri.squeeze().numpy(), affine, header)  
    nib.save(deformable_mri_nib, 'output/Feminad/Test_Landmarks/Deformable/Deformable_' + filename)  
    
    

    landmark = torch.from_numpy(nib.load(landmarks[i]).get_fdata()).unsqueeze(dim=0).unsqueeze(dim=0)
    get_landmarks_csv(landmarks[i],
                      "output/Feminad/Test_Landmarks/CSV/Original_" + filename + ".csv")
    
    affine_landmark = warp_nearest(landmark, affine_warp)
    affine_landmark_nib = nib.Nifti1Image(affine_landmark.squeeze().numpy(), affine, header)
    nib.save(affine_landmark_nib, 'output/Feminad/Test_Landmarks/Affine/Affine_Landmark_' + filename)
    get_landmarks_csv('output/Feminad/Test_Landmarks/Affine/Affine_Landmark_' + filename, 
                      "output/Feminad/Test_Landmarks/CSV/Affine_" + filename + ".csv")
    
    deformable_landmark = warp_nearest(affine_landmark, deformable_warp)
    deformable_landmark_nib = nib.Nifti1Image(deformable_landmark.squeeze().numpy(), affine, header)  
    nib.save(deformable_landmark_nib, 'output/Feminad/Test_Landmarks/Deformable/Deformable_Landmark_' + filename)  
    get_landmarks_csv('output/Feminad/Test_Landmarks/Deformable/Deformable_Landmark_' + filename, 
                      "output/Feminad/Test_Landmarks/CSV/Deformable_" + filename + ".csv")

    print('-'*100)


6516.nii.gz
tensor([[5.1962]], dtype=torch.float64)
tensor([[1.4142]], dtype=torch.float64)
tensor([[4.1231]], dtype=torch.float64)
----------------------------------------------------------------------------------------------------
6517.nii.gz
tensor([[2.2361]], dtype=torch.float64)
tensor([[1.]], dtype=torch.float64)
tensor([[1.4142]], dtype=torch.float64)
----------------------------------------------------------------------------------------------------
6518.nii.gz
tensor([[5.9161]], dtype=torch.float64)
tensor([[3.1623]], dtype=torch.float64)
tensor([[2.4495]], dtype=torch.float64)
----------------------------------------------------------------------------------------------------
6519.nii.gz
tensor([[1.4142]], dtype=torch.float64)
tensor([[1.4142]], dtype=torch.float64)
tensor([[1.4142]], dtype=torch.float64)
----------------------------------------------------------------------------------------------------
6520.nii.gz


the ground truth of class 0 is all 0, this may result in nan/inf distance.


tensor([[inf]], dtype=torch.float64)
tensor([[inf]], dtype=torch.float64)
tensor([[inf]], dtype=torch.float64)
----------------------------------------------------------------------------------------------------


the ground truth of class 0 is all 0, this may result in nan/inf distance.


In [196]:
template_csv = "output/Feminad/Test_Landmarks/CSV/Template.csv"
for i, mri_filename in enumerate(mris):
    filename = mri_filename.split('/')[-1].split('_')[-1]
    original_csv = "output/Feminad/Test_Landmarks/CSV/Original_" + filename + ".csv"
    affine_csv = "output/Feminad/Test_Landmarks/CSV/Affine_" + filename + ".csv"
    deformable_csv = "output/Feminad/Test_Landmarks/CSV/Deformable_" + filename + ".csv"

    original_tres = compute_csv_distance(original_csv, template_csv)
    affine_tres = compute_csv_distance(affine_csv, template_csv)
    deformable_tres = compute_csv_distance(deformable_csv, template_csv)
    for i in range(len(original_tres)):
        print(str(original_tres[i]) + ' - ' + str(affine_tres[i]) + ' - ' + str(deformable_tres[i]))
    print('Mean: ' + str(np.mean(original_tres)) + ' - ' + str(np.mean(affine_tres)) + ' - ' + str(np.mean(deformable_tres)))

    print('-'*45)

0.6235382907247959 - 0.16970562748477155 - 0.4907137658554119
0.6235382907247955 - 0.2683281572999742 - 0.05999999999999961
0.21213203435596428 - 0.29887288267756906 - 0.5198653671865439
0.6264183905346327 - 0.27495454169735045 - 0.34727510708370685
0.6235382907247955 - 0.2683281572999746 - 0.25455844122715726
0.78 - 0.2749545416973502 - 0.1935675171613252
Mean: 0.5815275495108306 - 0.25919065135949837 - 0.31099669975235744
---------------------------------------------
0.268328157299975 - 0.1200000000000001 - 0.1341640786499875
0.23999999999999932 - 0.0 - 0.146969384566991
0.4409081537009721 - 0.22449944320643644 - 0.22925391344315155
0.5091168824543137 - 0.2939387691339812 - 0.2545584412271569
0.379473319220205 - 0.16970562748477117 - 0.21633307652783912
0.3649657518178931 - 0.06000000000000005 - 0.06000000000000005
Mean: 0.3671320440822263 - 0.14469063997086481 - 0.17354648240252102
---------------------------------------------
0.7099295739719539 - 0.37947331922020544 - 0.25455844122